<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [ ]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
!pip install swmm5
import math
import os, sys
import swmm5
from pyswmm import Simulation, Links
from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
from google.colab import files
import pandas as pd

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 12.6 MB/s eta 0:00:00
  Created wheel for julian: filename=julian-0.14-py3-none-any.whl size=2613 sha256=630b3f0f2c045ce46e4d0ecd28b13cf45b83af3da94c9d72c62940ba208701ef
  Stored in directory: /root/.cache/pip/wheels/7c/c2/d4/08f8020a07dccace466111e011f3f38361cf57724f8e1f70bc
Successfully built julian
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.9/297.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.8/528.8 kB 27.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for swmm5: filename=swmm5-5.2.4-cp311-cp311-linux_x86_64.whl size=981440 sha256=d37a80860f12890f851a4b27860f98c53346d3d371a

# File Upload

In [ ]:
uploaded = files.upload()

inp = SwmmInput("Inlet_Drains_Model1.inp")
out = SwmmOutput("Inlet_Drains_Model1.out")
rpt = SwmmReport("Inlet_Drains_Model1.rpt")
available_sizes = files.upload("Available Sizes.txt")
print("Uploaded files:", os.listdir())
conduits_df = inp["CONDUITS"]
print(conduits_df)

Saving Available Sizes.txt to Available Sizes (1).txt


Uploaded files: ['.config', 'Inlet_Drains_Model1.rpt', 'Available Sizes.txt', 'Inlet_Drains_Model1.inp', 'Available Sizes (1).txt', 'Inlet_Drains_Model1.out', 'sample_data']
{'Street1': Conduit(name='Street1', from_node='Aux1', to_node='Aux2', length=377.31, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street2': Conduit(name='Street2', from_node='Aux2', to_node='Aux4', length=286.06, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street3': Conduit(name='Street3', from_node='Aux4', to_node='Aux5', length=239.41, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street4': Conduit(name='Street4', from_node='Aux5', to_node='Aux6', length=157.48, roughness=0.016, offset_upstream=0.0, offset_downstream=0.0, flow_initial=0.0, flow_max=nan), 'Street5': Conduit(name='Street5', from_node='Aux6', to_node='Aux7', length=526.0, roughness=0.016, offset_upstre

# Boundary Parameters

In [ ]:
peak_flow_dict = {}
peak_flow =  {}
peak_vel = {}
adjust_pipe_size = {}
available_sizes = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
vel_min = 0.8
vel_max = 5
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}

# Data Frame

In [ ]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']

# Store results
pipe_info = {}

# Accessing Data Dictionary

In [20]:
for conduit_name, conduit_data in inp["CONDUITS"].items():
  conduit_dict = vars(conduit_data) # Access all data in Conduit
  print (conduit_dict)

print(xsections)  # Print the content of the xsections InpSection object
for xsection_name, xsection_data in xsections.items():
  print(xsection_data)  # Print the name of each parameter in the xsections section

{'name': 'Street1', 'from_node': 'Aux1', 'to_node': 'Aux2', 'length': 377.31, 'roughness': 0.016, 'offset_upstream': 0.0, 'offset_downstream': 0.0, 'flow_initial': 0.0, 'flow_max': nan}
{'name': 'Street2', 'from_node': 'Aux2', 'to_node': 'Aux4', 'length': 286.06, 'roughness': 0.016, 'offset_upstream': 0.0, 'offset_downstream': 0.0, 'flow_initial': 0.0, 'flow_max': nan}
{'name': 'Street3', 'from_node': 'Aux4', 'to_node': 'Aux5', 'length': 239.41, 'roughness': 0.016, 'offset_upstream': 0.0, 'offset_downstream': 0.0, 'flow_initial': 0.0, 'flow_max': nan}
{'name': 'Street4', 'from_node': 'Aux5', 'to_node': 'Aux6', 'length': 157.48, 'roughness': 0.016, 'offset_upstream': 0.0, 'offset_downstream': 0.0, 'flow_initial': 0.0, 'flow_max': nan}
{'name': 'Street5', 'from_node': 'Aux6', 'to_node': 'Aux7', 'length': 526.0, 'roughness': 0.016, 'offset_upstream': 0.0, 'offset_downstream': 0.0, 'flow_initial': 0.0, 'flow_max': nan}
{'name': 'C3', 'from_node': 'J3', 'to_node': 'J4', 'length': 109.0, 'ro

# Data Gathering

In [21]:
with Simulation("Inlet_Drains_Model1.inp") as sim:
  for step in sim:
    pass

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  peak_flow = conduit_data.flow_max
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

  print (f"Section Name: {xsection_name}")
  print (f"Length: {length} m")
  print (f"Max Flow: {peak_flow} m3/s")
  print (f"Shape: {shape}")
  print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
  print (f"Inlet Offset: {in_offset} m")
  print (f"Outlet Offset: {out_offset} m")

Section Name: P8
Length: 377.31 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m
Section Name: P8
Length: 286.06 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m
Section Name: P8
Length: 239.41 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m
Section Name: P8
Length: 157.48 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m
Section Name: P8
Length: 526.0 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 0.0 m
Section Name: P8
Length: 109.0 m
Max Flow: nan m3/s
Shape: CIRCULAR
Geometry Parameters: 3.17 m, 0.0 m, 0.0 m, 0.0 m
Inlet Offset: 0.0 m
Outlet Offset: 6.0 m
Section Name: P8
Length: 133.0 m
Max Flow: nan m

# Optimization


Continuity Equation

In [ ]:
# Function for adjusting Pipe Sizes
def adjust_pipe_size (peak_flow, peak_vel):
    Area = peak_flow / peak_vel
    new_diameter = math.sqrt( Area / (0.25 * math.pi))
    inp.CONDUITS.__sizeof__ = new_diameter
    pass

Construction Cost

In [ ]:
# Function for the Computation of Construction Costs
def construction_costs (unit_cost, length, size):
  pass

Identifying the Largest Pipe Size and Minimum Slope

In [14]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size(available_sizes):
  largest_size = 0.0
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

available_sizes = "Available Sizes.txt"
largest_size = largest_size(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

Largest Pipe Size: 1.8 m


Initializing Pipe Sizes and Slopes

In [ ]:
# Set all pipe sizes to largest_size
for xsection_name, xsection_data in inp["CONDUITS"].items():
  xsection_data.height = largest_size(available_sizes)

# Saving data before Simulation
inp.write_file("Modified_Inlet_Drains_Model1.inp")

Simulation of Data before Optimization

In [ ]:
# Simulation of Data
with Simulation("Modified_Inlet_Drains_Model1.inp") as sim:
  for step in sim:
    pass

Optimization of Pipe Sizes and Slopes

In [ ]:
# Optimization of Sizes and Slopes
for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  peak_flow = conduit_data.flow_max
  peak_vel = peak_flow / length

  for xsection_name, xsection_data in inp["CONDUITS"].items():
    shape = xsection_data.shape
    geom1 = xsection_data.height
    geom2 = xsection_data.parameter_2
    geom3 = xsection_data.parameter_3
    geom4 = xsection_data.parameter_4